In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None)
import plotly_express as px
from itertools import groupby

In [4]:
# instead of scraping results, use ergast
results = pd.read_csv('./f1db_csv/results.csv')
races = pd.read_csv('./f1db_csv/races.csv')
status = pd.read_csv('./f1db_csv/status.csv')
drivers = pd.read_csv('./f1db_csv/drivers.csv')
constructors = pd.read_csv('./f1db_csv/constructors.csv')
circuits = pd.read_csv('./f1db_csv/circuits.csv')

results_info = results.merge(races,on='raceId').merge(drivers,on='driverId').merge(constructors,on='constructorId').merge(circuits,on='circuitId')
results_sel = results_info[['raceId','circuitId','driverId','code','forename','surname','constructorId','name_y','year','round','name_x','statusId','points','name']]
results_status = results_sel.merge(status,on='statusId')
results_status.columns = ['raceId', 'circuitId','driverId', 'code', 'forename', 'surname', 'constructorId', 'constructor','year', 'round', 'race', 'statusId','points', 'circuit','status']
results_status['driver'] = results_status['forename'] + " " + results_status['surname']
results_status.head()

/Users/blossomtang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'url_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,raceId,circuitId,driverId,code,forename,surname,constructorId,constructor,year,round,race,statusId,points,circuit,status,driver
0,18,1,1,HAM,Lewis,Hamilton,1,McLaren,2008,1,Australian Grand Prix,1,10.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton
1,36,1,1,HAM,Lewis,Hamilton,1,McLaren,2007,1,Australian Grand Prix,1,6.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton
2,338,1,1,HAM,Lewis,Hamilton,1,McLaren,2010,2,Australian Grand Prix,1,8.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton
3,841,1,1,HAM,Lewis,Hamilton,1,McLaren,2011,1,Australian Grand Prix,1,18.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton
4,860,1,1,HAM,Lewis,Hamilton,1,McLaren,2012,1,Australian Grand Prix,1,15.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton


In [170]:
results_status.to_csv('./data/results_status.csv',index=False)

# analyse accident rate with weather conditions

In [47]:
# accident status codes
acc_statusid = [3,4,20,130]
# acc_statusid = [3,4,20,73,82,89,100,104,107,130] # mainly accident, collision, spun off, collision damage
# acc_statusid = [73,82,89,100,104,107] # injured, injury, safety concerns, driver unwell, fatal accident, eye injury have few results.
status[status['statusId'].isin(acc_statusid)]

,statusId,status
2,3,Accident
3,4,Collision
19,20,Spun off
127,130,Collision damage


In [48]:
results_status['accident'] = results_status['statusId'].isin(acc_statusid)
results_status.head()

,raceId,circuitId,driverId,code,forename,surname,constructorId,constructor,year,round,race,statusId,points,circuit,status,driver,accident
0,18,1,1,HAM,Lewis,Hamilton,1,McLaren,2008,1,Australian Grand Prix,1,10.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False
1,36,1,1,HAM,Lewis,Hamilton,1,McLaren,2007,1,Australian Grand Prix,1,6.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False
2,338,1,1,HAM,Lewis,Hamilton,1,McLaren,2010,2,Australian Grand Prix,1,8.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False
3,841,1,1,HAM,Lewis,Hamilton,1,McLaren,2011,1,Australian Grand Prix,1,18.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False
4,860,1,1,HAM,Lewis,Hamilton,1,McLaren,2012,1,Australian Grand Prix,1,15.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False


In [49]:
# individual race and weather
weather = pd.read_csv('./data/weather.csv')
accidents_weather_indiv = results_status.merge(weather,on=['year','round'])
accidents_weather_indiv.head()

,raceId,circuitId,driverId,code,forename,surname,constructorId,constructor,year,round,race,statusId,points,circuit,status,driver,accident,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed
0,989,1,4,ALO,Fernando,Alonso,1,McLaren,2018,1,Australian Grand Prix,1,10.0,Albert Park Grand Prix Circuit,Finished,Fernando Alonso,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
1,989,1,838,VAN,Stoffel,Vandoorne,1,McLaren,2018,1,Australian Grand Prix,1,2.0,Albert Park Grand Prix Circuit,Finished,Stoffel Vandoorne,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
2,989,1,1,HAM,Lewis,Hamilton,131,Mercedes,2018,1,Australian Grand Prix,1,18.0,Albert Park Grand Prix Circuit,Finished,Lewis Hamilton,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
3,989,1,822,BOT,Valtteri,Bottas,131,Mercedes,2018,1,Australian Grand Prix,1,4.0,Albert Park Grand Prix Circuit,Finished,Valtteri Bottas,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
4,989,1,840,STR,Lance,Stroll,3,Williams,2018,1,Australian Grand Prix,1,0.0,Albert Park Grand Prix Circuit,Finished,Lance Stroll,False,24.077477,30.915315,997.003604,True,36.324324,3.691892


In [50]:
acc_count = results_status.groupby(['raceId','race','year','round']).agg({'accident':'sum'}).reset_index()
acc_count[acc_count['accident']>7]

,raceId,race,year,round,accident
17,18,Australian Grand Prix,2008,1,8
61,62,United States Grand Prix,2006,10,10
109,110,Brazilian Grand Prix,2003,3,8
123,124,Australian Grand Prix,2002,1,9
170,171,Italian Grand Prix,2000,14,9
198,199,British Grand Prix,1998,9,10
202,203,Belgian Grand Prix,1998,13,12
210,211,Monaco Grand Prix,1997,5,12
228,229,Monaco Grand Prix,1996,6,13
229,230,Spanish Grand Prix,1996,7,10


In [51]:
acc_count.to_csv("./data/accidents.csv",index=False)

In [52]:
# combine accidents and weather
weather = pd.read_csv('./data/weather.csv')
accidents_weather = acc_count.merge(weather,on=['year','round'])

accidents_weather.head()

,raceId,race,year,round,accident,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed
0,989,Australian Grand Prix,2018,1,0,24.077477,30.915315,997.003604,True,36.324324,3.691892
1,990,Bahrain Grand Prix,2018,2,0,27.982524,47.363107,1009.494175,False,32.198058,0.958252
2,991,Chinese Grand Prix,2018,3,1,19.446429,24.089286,1018.131250,True,37.019643,1.837500
3,992,Azerbaijan Grand Prix,2018,4,6,16.661404,45.651754,1021.913158,False,25.251754,2.222807
4,993,Spanish Grand Prix,2018,5,3,16.050476,52.286667,1001.541905,True,32.339048,1.952381


In [53]:
accidents_weather.to_csv('./data/accidents_weather.csv',index=False)

In [144]:
px.violin(accidents_weather,x='year',y='accident',color='Rainfall',hover_data=['round'],points='outliers',box=True)

In [157]:
px.violin(accidents_weather,y='accident',color='Rainfall',hover_data=['race','year','round'],points='all')

In [176]:
# count statuses by year
results_status[(results_status['year'] > 1950) & results_status['statusId'].isin(acc_statusid)].groupby(['year','statusId','status']).count()

raceId  circuitId  driverId  code  forename  \
year statusId status                                                          
1951 3        Accident               9          9         9     9         9   
     20       Spun off               1          1         1     1         1   
1952 3        Accident               5          5         5     5         5   
     20       Spun off               1          1         1     1         1   
1953 3        Accident              13         13        13    13        13   
...                                ...        ...       ...   ...       ...   
2021 130      Collision damage       3          3         3     3         3   
2022 3        Accident               5          5         5     5         5   
     4        Collision              6          6         6     6         6   
     20       Spun off               1          1         1     1         1   
     130      Collision damage       6          6         6     6         6   

                                surname  constructorId  constructor  round  \
year statusId status                                                         
1951 3        Accident                9              9            9      9   
     20       Spun off                1              1            1      1   
1952 3        Accident                5              5            5      5   
     20       Spun off                1              1            1      1   
1953 3        Accident               13             13           13     13   
...                                 ...            ...          ...    ...   
2021 130      Collision damage        3              3            3      3   
2022 3        Accident                5              5            5      5   
     4        Collision               6              6            6      6   
     20       Spun off                1              1            1      1   
     130      Collision damage        6              6            6      6   

                                race  points  driver  accident  
year statusId status                                            
1951 3        Accident             9       9       9         9  
     20       Spun off             1       1       1         1  
1952 3        Accident             5       5       5         5  
     20       Spun off             1       1       1         1  
1953 3        Accident            13      13      13        13  
...                              ...     ...     ...       ...  
2021 130      Collision damage     3       3       3         3  
2022 3        Accident             5       5       5         5  
     4        Collision            6       6       6         6  
     20       Spun off             1       1       1         1  
     130      Collision damage     6       6       6         6  

[190 rows x 13 columns]

In [91]:
# check races with particular statuses
results_status[results_status['status'] == 'Spun off']

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver,accident
11611,51,1,Lewis,Hamilton,1,McLaren,2007,16,Chinese Grand Prix,20,Spun off,Lewis Hamilton,True
11612,2,5,Heikki,Kovalainen,1,McLaren,2009,2,Malaysian Grand Prix,20,Spun off,Heikki Kovalainen,True
11613,135,8,Kimi,Räikkönen,1,McLaren,2002,12,German Grand Prix,20,Spun off,Kimi Räikkönen,True
11614,188,14,David,Coulthard,1,McLaren,1999,14,European Grand Prix,20,Spun off,David Coulthard,True
11615,199,14,David,Coulthard,1,McLaren,1998,9,British Grand Prix,20,Spun off,David Coulthard,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12397,819,663,Ken,Wharton,148,Frazer Nash,1952,3,Belgian Grand Prix,20,Spun off,Ken Wharton,True
12398,835,797,Jackie,Holmes,159,Olson,1950,3,Indianapolis 500,20,Spun off,Jackie Holmes,True
12399,1020,815,Sergio,Pérez,211,Racing Point,2019,11,German Grand Prix,20,Spun off,Sergio Pérez,True
12400,957,836,Pascal,Wehrlein,209,Manor Marussia,2016,10,British Grand Prix,20,Spun off,Pascal Wehrlein,True


In [53]:
# look up race results by raceid
results_status[results_status['raceId'] == 1024]

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver
831,1024,832,Carlos,Sainz,1,McLaren,2019,15,Singapore Grand Prix,1,Finished,Carlos Sainz
857,1024,846,Lando,Norris,1,McLaren,2019,15,Singapore Grand Prix,1,Finished,Lando Norris
1022,1024,1,Lewis,Hamilton,131,Mercedes,2019,15,Singapore Grand Prix,1,Finished,Lewis Hamilton
1286,1024,822,Valtteri,Bottas,131,Mercedes,2019,15,Singapore Grand Prix,1,Finished,Valtteri Bottas
1486,1024,9,Robert,Kubica,3,Williams,2019,15,Singapore Grand Prix,1,Finished,Robert Kubica
2667,1024,807,Nico,Hülkenberg,4,Renault,2019,15,Singapore Grand Prix,1,Finished,Nico Hülkenberg
2690,1024,817,Daniel,Ricciardo,4,Renault,2019,15,Singapore Grand Prix,1,Finished,Daniel Ricciardo
3136,1024,20,Sebastian,Vettel,6,Ferrari,2019,15,Singapore Grand Prix,1,Finished,Sebastian Vettel
3925,1024,844,Charles,Leclerc,6,Ferrari,2019,15,Singapore Grand Prix,1,Finished,Charles Leclerc
4221,1024,826,Daniil,Kvyat,5,Toro Rosso,2019,15,Singapore Grand Prix,1,Finished,Daniil Kvyat


In [43]:
# look up race results by year/round
results_status[(results_status['year'] == 2021) & (results_status['round'] == 1)].sort_values('constructor')

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver
8818,1052,841,Antonio,Giovinazzi,51,Alfa Romeo,2021,1,Bahrain Grand Prix,11,+1 Lap,Antonio Giovinazzi
4283,1052,8,Kimi,Räikkönen,51,Alfa Romeo,2021,1,Bahrain Grand Prix,1,Finished,Kimi Räikkönen
17251,1052,842,Pierre,Gasly,213,AlphaTauri,2021,1,Bahrain Grand Prix,31,Retired,Pierre Gasly
6945,1052,852,Yuki,Tsunoda,213,AlphaTauri,2021,1,Bahrain Grand Prix,1,Finished,Yuki Tsunoda
13657,1052,4,Fernando,Alonso,214,Alpine F1 Team,2021,1,Bahrain Grand Prix,23,Brakes,Fernando Alonso
8427,1052,839,Esteban,Ocon,214,Alpine F1 Team,2021,1,Bahrain Grand Prix,11,+1 Lap,Esteban Ocon
9433,1052,20,Sebastian,Vettel,117,Aston Martin,2021,1,Bahrain Grand Prix,11,+1 Lap,Sebastian Vettel
5500,1052,840,Lance,Stroll,117,Aston Martin,2021,1,Bahrain Grand Prix,1,Finished,Lance Stroll
3940,1052,844,Charles,Leclerc,6,Ferrari,2021,1,Bahrain Grand Prix,1,Finished,Charles Leclerc
3885,1052,832,Carlos,Sainz,6,Ferrari,2021,1,Bahrain Grand Prix,1,Finished,Carlos Sainz


In [48]:
results_status[results_status['status'] == '+1 Lap'].sort_values(['year'])

,raceId,driverId,forename,surname,constructorId,constructor,round,race,statusId,status,driver
year,,,,,,,,,,,
1950,6,6,6,6,6,6,6,6,6,6,6
1951,7,7,7,7,7,7,7,7,7,7,7
1952,8,8,8,8,8,8,8,8,8,8,8
1953,23,23,23,23,23,23,23,23,23,23,23
1954,19,19,19,19,19,19,19,19,19,19,19
1955,12,12,12,12,12,12,12,12,12,12,12
1956,9,9,9,9,9,9,9,9,9,9,9
1957,10,10,10,10,10,10,10,10,10,10,10
1958,16,16,16,16,16,16,16,16,16,16,16


# tyrelife

In [44]:
tyre_rank = ['UNKNOWN','WET','INTERMEDIATE','HYPERSOFT','ULTRASOFT','SUPERSOFT','SOFT','MEDIUM','HARD','SUPERHARD']
tyre_rank_num = ['0 UNKNOWN','1 WET','2 INTERMEDIATE','3 HYPERSOFT','4 ULTRASOFT','5 SUPERSOFT','6 SOFT','7 MEDIUM','8 HARD','9 SUPERHARD']

# tyre_rank_dict = {x:i for i,x in enumerate(tyre_rank)}
tyre_rank_dict = {x:tyre_rank_num[i] for i,x in enumerate(tyre_rank)}

tyre_rank_dict

{'UNKNOWN': '0 UNKNOWN',
 'WET': '1 WET',
 'INTERMEDIATE': '2 INTERMEDIATE',
 'HYPERSOFT': '3 HYPERSOFT',
 'ULTRASOFT': '4 ULTRASOFT',
 'SUPERSOFT': '5 SUPERSOFT',
 'SOFT': '6 SOFT',
 'MEDIUM': '7 MEDIUM',
 'HARD': '8 HARD',
 'SUPERHARD': '9 SUPERHARD'}

In [45]:
tyrelife = pd.read_csv('./data/tyrelife.csv')
tyrelife['Compound_num'] = tyrelife['Compound'].map(tyre_rank_dict)#.astype(int)
tyrelife.tail(20)

,Team,Driver,Stint,Compound,TyreLife,round,year,Compound_num
4855,Haas F1 Team,MSC,2.0,MEDIUM,7.0,17,2022,7 MEDIUM
4856,Haas F1 Team,MSC,3.0,SOFT,17.0,17,2022,6 SOFT
4857,McLaren,NOR,1.0,INTERMEDIATE,36.0,17,2022,2 INTERMEDIATE
4858,McLaren,NOR,2.0,MEDIUM,23.0,17,2022,7 MEDIUM
4859,McLaren,RIC,1.0,INTERMEDIATE,36.0,17,2022,2 INTERMEDIATE
4860,McLaren,RIC,2.0,SOFT,23.0,17,2022,6 SOFT
4861,Mercedes,HAM,1.0,INTERMEDIATE,34.0,17,2022,2 INTERMEDIATE
4862,Mercedes,HAM,2.0,MEDIUM,25.0,17,2022,7 MEDIUM
4863,Mercedes,RUS,2.0,INTERMEDIATE,21.0,17,2022,2 INTERMEDIATE
4864,Mercedes,RUS,3.0,MEDIUM,14.0,17,2022,7 MEDIUM


In [46]:
# tyrelife2 uses numbered tyre hardness
tyrelife.to_csv('./data/tyrelife2.csv',index=False)

In [40]:
tyrelife[(tyrelife['year']==2021) & (tyrelife['round']==1)]

,Team,Driver,Stint,Compound,TyreLife,round,year,Compound_num
2957,Alfa Romeo Racing,GIO,1.0,MEDIUM,12.0,1,2021,7
2958,Alfa Romeo Racing,GIO,2.0,HARD,18.0,1,2021,8
2959,Alfa Romeo Racing,GIO,3.0,MEDIUM,25.0,1,2021,7
2960,Alfa Romeo Racing,RAI,1.0,MEDIUM,13.0,1,2021,7
2961,Alfa Romeo Racing,RAI,2.0,HARD,16.0,1,2021,8
2962,Alfa Romeo Racing,RAI,3.0,MEDIUM,NaN,1,2021,7
2963,AlphaTauri,GAS,1.0,MEDIUM,7.0,1,2021,7
2964,AlphaTauri,GAS,2.0,HARD,15.0,1,2021,8
2965,AlphaTauri,GAS,3.0,MEDIUM,23.0,1,2021,7
2966,AlphaTauri,GAS,4.0,HARD,13.0,1,2021,8


In [39]:
# get tyrelife laps per race
race_laps = tyrelife.groupby(['round','year','Driver']).agg({'TyreLife':'sum'}).reset_index().groupby(['round','year']).agg({'TyreLife':'median'}).reset_index()

,round,year,TyreLife
0,1,2018,59.0
1,1,2019,57.5
2,1,2020,71.0
3,1,2021,56.5
4,2,2018,57.0
5,2,2019,60.0
6,2,2020,70.0
7,2,2021,68.0
8,3,2018,107.0
9,3,2019,55.0


In [199]:
# median tyre life per constructor
tyrelife.groupby(['Team','Compound']).agg({'TyreLife':'median'}).reset_index()

,Team,Compound,TyreLife
0,Alfa Romeo Racing,HARD,32.0
1,Alfa Romeo Racing,INTERMEDIATE,19.0
2,Alfa Romeo Racing,MEDIUM,20.0
3,Alfa Romeo Racing,SOFT,16.0
4,Alfa Romeo Racing,UNKNOWN,2.0
...,...,...,...
108,Williams,SOFT,15.0
109,Williams,SUPERSOFT,21.0
110,Williams,ULTRASOFT,13.0
111,Williams,UNKNOWN,1.0


In [54]:
# tyre sequence
tyrelist = tyrelife.sort_values(['year','round','Team','Driver','Stint']).groupby(['year','round','Team','Driver']).agg({'Compound':list,'Compound_num':list}).reset_index()

constructor_rename_dict = {'Red Bull':'Red Bull Racing','Alfa Romeo':'Alfa Romeo Racing','Alpine F1 Team':'Alpine'}
results_status['constructor'] = results_status['constructor'].replace(constructor_rename_dict)

tyrelist_status = (tyrelist.merge(results_status,left_on=['year','round','Team','Driver'],right_on=['year','round','constructor','code'])
.merge(weather[['year','round','Rainfall']],on=['year','round'],how='left').drop_duplicates(subset=['year','round','Team','Driver']))
# drop duplicates
tyrelist_status['Compound'] = tyrelist_status['Compound'].apply(lambda x: [key for key, _group in groupby(x)])

In [61]:
tyrelist.dtypes

year             int64
round            int64
Team            object
Driver          object
Compound        object
Compound_num    object
dtype: object

In [62]:
tyrelist[(tyrelist['year']==2022) & (tyrelist['round']==17)]

,year,round,Team,Driver,Compound,Compound_num
1815,2022,17,Alfa Romeo,BOT,"[INTERMEDIATE, SOFT]","[2 INTERMEDIATE, 6 SOFT]"
1816,2022,17,Alfa Romeo,ZHO,[INTERMEDIATE],[2 INTERMEDIATE]
1817,2022,17,AlphaTauri,GAS,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"
1818,2022,17,AlphaTauri,TSU,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"
1819,2022,17,Alpine,ALO,[INTERMEDIATE],[2 INTERMEDIATE]
1820,2022,17,Alpine,OCO,[INTERMEDIATE],[2 INTERMEDIATE]
1821,2022,17,Aston Martin,STR,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"
1822,2022,17,Aston Martin,VET,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"
1823,2022,17,Ferrari,LEC,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"
1824,2022,17,Ferrari,SAI,"[INTERMEDIATE, MEDIUM]","[2 INTERMEDIATE, 7 MEDIUM]"


In [57]:
tyrelist_status[(tyrelist_status['year']==2022) & (tyrelist_status['round']==17)]

,year,round,Team,Driver,Compound,Compound_num,raceId,circuitId,driverId,code,forename,surname,constructorId,constructor,race,statusId,points,circuit,status,driver,accident,Rainfall


In [196]:
tyrelist_status.to_csv('./data/tyrelist_status.csv',index=False)

In [34]:
# count number of times strategy used
tyrelist_count = tyrelist.groupby(['year',tyrelist['Compound'].map(tuple),tyrelist['Compound_num'].map(tuple)]).agg({'round':'count'}).reset_index()
tyrelist_count

,year,Compound,Compound_num,round
0,2018,"(HYPERSOFT,)","(3,)",3
1,2018,"(HYPERSOFT, HYPERSOFT, ULTRASOFT, SUPERSOFT)","(3, 3, 4, 5)",1
2,2018,"(HYPERSOFT, SOFT)","(3, 6)",11
3,2018,"(HYPERSOFT, SOFT, ULTRASOFT)","(3, 6, 4)",1
4,2018,"(HYPERSOFT, SUPERSOFT)","(3, 5)",19
...,...,...,...,...
291,2021,"(SOFT, SOFT, SOFT, MEDIUM)","(6, 6, 6, 7)",5
292,2021,"(WET,)","(1,)",20
293,2021,"(WET, INTERMEDIATE, MEDIUM, MEDIUM, MEDIUM)","(1, 2, 7, 7, 7)",1
294,2021,"(WET, INTERMEDIATE, MEDIUM, SOFT, MEDIUM, MEDIUM)","(1, 2, 7, 6, 7, 7)",1


In [208]:
tyrelist_count.to_csv('./data/tyrelist_count.csv',index=False)

# circuits

In [92]:
downforce = pd.read_csv('./data/downforce.csv')
downforce.drop(['lat','lng','alt','url'],axis=1,inplace=True)
downforce.to_csv('./data/downforce.csv',index=False)

# telemetry

In [122]:
telemetry = pd.read_csv('./data/telemetry.csv')
telemetry.head()

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,round,year,driver
0,2021-03-28 16:26:05.207,0 days 01:56:05.197000,NaN,517.434722,0 days 00:00:00,11321,279,7,100,False,0,interpolation,0.059691,0.000011,OnTrack,-383,1191,-158,1,2021,SAI
1,2021-03-28 16:26:05.220,0 days 01:56:05.210000,NaN,517.434722,0 days 00:00:00.013000,11326,280,7,100,False,0,car,1.074167,0.000199,OnTrack,-383,1202,-159,1,2021,SAI
2,2021-03-28 16:26:05.334,0 days 01:56:05.324000,NaN,517.434722,0 days 00:00:00.127000,11370,281,7,100,False,0,pos,9.987024,0.001847,OnTrack,-379,1291,-159,1,2021,SAI
3,2021-03-28 16:26:05.459,0 days 01:56:05.449000,3.0,517.434722,0 days 00:00:00.252000,11415,282,7,100,False,0,car,19.795833,0.003660,OnTrack,-373,1388,-158,1,2021,SAI
4,2021-03-28 16:26:05.554,0 days 01:56:05.544000,3.0,517.668056,0 days 00:00:00.347000,11463,283,7,100,False,0,pos,27.277017,0.005043,OnTrack,-370,1463,-158,1,2021,SAI


In [189]:
telemetry_circuit = telemetry.merge(results_status[['year','round','circuit']],on=['year','round'])
telemetry_circuit.to_csv('./data/telemetry_circuit.csv',index=False)

In [127]:
gearshifts = telemetry.groupby(['year','round','driver']).agg({'nGear':list}).reset_index()
gearshifts['nGear'] = gearshifts['nGear'].apply(lambda x: [key for key, _group in groupby(x)])
gearshifts

,year,round,driver,nGear
0,2021,1,ALO,"[7, 8, 6, 4, 3, 2, 3, 4, 5, 6, 7, 5, 4, 3, 4, 5, 6, 5, 4, 3, 2, 3, 4, 5, 6, 4, 3, 2, 3, 4, 5, 6, 7, 6, 5, 4, 5, 6, 5, 4, 3, 4, 5, 6, 7, 6, 4, 3, 4, 5, 6, 7]"
1,2021,1,BOT,"[7, 8, 7, 4, 3, 4, 5, 6, 7, 5, 4, 5, 6, 5, 6, 5, 3, 2, 3, 4, 5, 6, 5, 3, 4, 5, 6, 7, 8, 7, 5, 6, 7, 6, 5, 4, 5, 6, 7, 6, 4, 5, 6, 7]"
2,2021,1,GAS,"[7, 8, 6, 4, 2, 3, 4, 5, 6, 7, 8, 6, 3, 4, 5, 6, 5, 6, 4, 2, 3, 4, 5, 6, 5, 4, 3, 2, 3, 4, 5, 6, 7, 8, 5, 4, 5, 6, 5, 4, 5, 6, 7, 8, 7, 5, 3, 4, 5, 6, 7]"
3,2021,1,GIO,"[8, 7, 6, 4, 3, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 5, 6, 4, 3, 2, 3, 4, 5, 6, 5, 4, 3, 2, 3, 4, 5, 6, 7, 6, 5, 4, 5, 6, 5, 4, 5, 6, 7, 6, 5, 4, 5, 6, 7]"
4,2021,1,HAM,"[8, 6, 4, 3, 2, 3, 4, 5, 6, 7, 6, 5, 4, 5, 6, 5, 6, 5, 3, 4, 5, 6, 5, 4, 3, 4, 5, 6, 7, 6, 5, 4, 5, 6, 7, 6, 5, 4, 5, 6, 7, 6, 4, 3, 4, 5, 6, 7]"
...,...,...,...,...
340,2021,22,SAI,"[5, 6, 7, 5, 4, 5, 6, 7, 5, 4, 3, 4, 5, 6, 7, 8, 7, 4, 3, 2, 3, 4, 5, 6, 7, 8, 6, 5, 4, 5, 6, 7, 4, 3, 4, 5, 6, 7, 5, 3, 4, 5]"
341,2021,22,STR,"[6, 7, 6, 4, 5, 6, 7, 8, 7, 5, 3, 4, 5, 6, 7, 8, 7, 5, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 6, 7, 6, 5, 4, 3, 4, 5, 6, 5, 4, 5]"
342,2021,22,TSU,"[5, 6, 7, 6, 4, 5, 6, 7, 5, 3, 4, 5, 6, 7, 8, 7, 4, 2, 3, 4, 5, 6, 7, 8, 6, 5, 4, 5, 6, 7, 6, 4, 3, 4, 5, 6, 4, 3, 4, 5]"
343,2021,22,VER,"[5, 6, 7, 6, 5, 4, 5, 6, 7, 8, 7, 5, 3, 4, 5, 6, 7, 8, 7, 5, 3, 2, 3, 4, 5, 6, 7, 8, 6, 5, 6, 7, 4, 3, 4, 5, 6, 5, 4, 5]"


In [150]:
gearshifts['gearshifts'] = gearshifts['nGear'].apply(len)
gearshift_count = gearshifts.groupby(['year','round']).agg({'gearshifts':'median'}).reset_index()

In [132]:
# add to combined table
tyrelist_status.merge(gearshift_count,on=['year','round'],how='left')

,year,round,Team,Driver,Compound,Compound_num,raceId,driverId,code,forename,surname,constructorId,constructor,race,statusId,points,status,driver,accident,Rainfall,gearshifts
0,2018,1,Ferrari,RAI,"[ULTRASOFT, SOFT]","[4 ULTRASOFT, 6 SOFT]",989,8,RAI,Kimi,Räikkönen,6,Ferrari,Australian Grand Prix,1,15.0,Finished,Kimi Räikkönen,False,True,NaN
1,2018,1,Ferrari,VET,"[ULTRASOFT, SOFT, ULTRASOFT]","[4 ULTRASOFT, 6 SOFT, 4 ULTRASOFT]",989,20,VET,Sebastian,Vettel,6,Ferrari,Australian Grand Prix,1,25.0,Finished,Sebastian Vettel,False,True,NaN
2,2018,1,Force India,OCO,"[ULTRASOFT, SOFT]","[4 ULTRASOFT, 6 SOFT]",989,839,OCO,Esteban,Ocon,10,Force India,Australian Grand Prix,1,0.0,Finished,Esteban Ocon,False,True,NaN
3,2018,1,Force India,PER,"[ULTRASOFT, SOFT]","[4 ULTRASOFT, 6 SOFT]",989,815,PER,Sergio,Pérez,10,Force India,Australian Grand Prix,1,0.0,Finished,Sergio Pérez,False,True,NaN
4,2018,1,Haas F1 Team,GRO,[ULTRASOFT],[4 ULTRASOFT],989,154,GRO,Romain,Grosjean,210,Haas F1 Team,Australian Grand Prix,36,0.0,Wheel,Romain Grosjean,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2021,22,Mercedes,HAM,"[MEDIUM, HARD]","[7 MEDIUM, 8 HARD]",1073,1,HAM,Lewis,Hamilton,131,Mercedes,Abu Dhabi Grand Prix,1,18.0,Finished,Lewis Hamilton,False,False,40.0
1515,2021,22,Red Bull Racing,PER,"[SOFT, HARD, SOFT]","[6 SOFT, 8 HARD, 8 HARD, 6 SOFT]",1073,815,PER,Sergio,Pérez,9,Red Bull Racing,Abu Dhabi Grand Prix,5,0.0,Engine,Sergio Pérez,False,False,40.0
1516,2021,22,Red Bull Racing,VER,"[SOFT, HARD, SOFT]","[6 SOFT, 8 HARD, 8 HARD, 6 SOFT]",1073,830,VER,Max,Verstappen,9,Red Bull Racing,Abu Dhabi Grand Prix,1,26.0,Finished,Max Verstappen,False,False,40.0
1517,2021,22,Williams,LAT,"[MEDIUM, HARD]","[7 MEDIUM, 8 HARD]",1073,849,LAT,Nicholas,Latifi,3,Williams,Abu Dhabi Grand Prix,3,0.0,Accident,Nicholas Latifi,True,False,40.0


In [163]:
# or simpler circuit info
races = pd.read_csv('./f1db_csv/races.csv')[['year','round','name','date','circuitId']]
downforce = pd.read_csv('./data/downforce.csv')
circuit_info = (
    gearshift_count
    .merge(races,on=['year','round'])
    .merge(weather[['Rainfall','year','round']],on=['year','round'],how='left').drop_duplicates()
    .merge(downforce[['circuitId','circuitName','downforce']],on='circuitId')
    )
circuit_info

,year,round,gearshifts,name,date,circuitId,Rainfall,circuitName,downforce
0,2021,1,48.0,Bahrain Grand Prix,2021-03-28,3,False,Bahrain International Circuit,2.0
1,2021,2,38.0,Emilia Romagna Grand Prix,2021-04-18,21,True,Autodromo Enzo e Dino Ferrari,4.0
2,2021,3,40.0,Portuguese Grand Prix,2021-05-02,75,False,Autódromo Internacional do Algarve,NaN
3,2021,4,36.0,Spanish Grand Prix,2021-05-09,4,False,Circuit de Barcelona-Catalunya,4.0
4,2021,5,42.0,Monaco Grand Prix,2021-05-23,6,False,Circuit de Monaco,5.0
5,2021,6,57.0,Azerbaijan Grand Prix,2021-06-06,73,False,Baku City Circuit,2.0
6,2021,7,44.0,French Grand Prix,2021-06-20,34,False,Circuit Paul Ricard,NaN
7,2021,8,36.0,Styrian Grand Prix,2021-06-27,70,False,Red Bull Ring,3.0
8,2021,9,35.0,Austrian Grand Prix,2021-07-04,70,True,Red Bull Ring,3.0
9,2021,11,42.0,Hungarian Grand Prix,2021-08-01,11,True,Hungaroring,4.0


In [164]:
circuit_info.sort_values('gearshifts')

,year,round,gearshifts,name,date,circuitId,Rainfall,circuitName,downforce
10,2021,13,34.0,Dutch Grand Prix,2021-09-05,39,False,Circuit Park Zandvoort,2.0
8,2021,9,35.0,Austrian Grand Prix,2021-07-04,70,True,Red Bull Ring,3.0
15,2021,20,35.0,Qatar Grand Prix,2021-11-21,78,True,Losail International Circuit,NaN
14,2021,18,35.0,Mexico City Grand Prix,2021-11-07,32,False,Autódromo Hermanos Rodríguez,4.0
12,2021,16,35.5,Turkish Grand Prix,2021-10-10,5,False,Istanbul Park,NaN
3,2021,4,36.0,Spanish Grand Prix,2021-05-09,4,False,Circuit de Barcelona-Catalunya,4.0
7,2021,8,36.0,Styrian Grand Prix,2021-06-27,70,False,Red Bull Ring,3.0
1,2021,2,38.0,Emilia Romagna Grand Prix,2021-04-18,21,True,Autodromo Enzo e Dino Ferrari,4.0
16,2021,21,38.0,Saudi Arabian Grand Prix,2021-12-05,77,False,Jeddah Corniche Circuit,NaN
17,2021,22,40.0,Abu Dhabi Grand Prix,2021-12-12,24,False,Yas Marina Circuit,3.0


# budgets and points

In [231]:
budget = pd.read_csv('./data/budgets.csv')
budget.head()

,team,year,budget (USD$M)
0,Mercedes,2015,527.6
1,Mercedes,2016,352.0
2,Mercedes,2017,352.1
3,Mercedes,2018,400.0
4,Mercedes,2019,484.0


In [232]:
results_annual = results_status.groupby(['year','constructor']).agg({'points':'sum'}).reset_index()
budget_subset = budget[budget['year'].isin([2015,2016,2017,2018,2019])]
budget_points = budget_subset.merge(results_annual,left_on=['team','year'],right_on=['constructor','year'],how='left')
budget_points.head(100)

,team,year,budget (USD$M),constructor,points
0,Mercedes,2015,527.6,Mercedes,703.0
1,Mercedes,2016,352.0,Mercedes,765.0
2,Mercedes,2017,352.1,Mercedes,668.0
3,Mercedes,2018,400.0,Mercedes,655.0
4,Mercedes,2019,484.0,Mercedes,739.0
5,Ferrari,2015,474.7,Ferrari,428.0
6,Ferrari,2016,483.3,Ferrari,398.0
7,Ferrari,2017,295.3,Ferrari,522.0
8,Ferrari,2018,410.0,Ferrari,571.0
9,Ferrari,2019,463.0,Ferrari,504.0


In [239]:
budget_points.to_csv('./data/budget_points.csv',index=False)

# pit stop

In [7]:
pitstop = pd.read_csv("./f1db_csv/pit_stops.csv")
sprint_results = pd.read_csv("./f1db_csv/sprint_results.csv")
pitstop.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [34]:
pitstop_comb = (results
.merge(constructors,on="constructorId")
.merge(pitstop,on=['raceId','driverId'])
.merge(races,on='raceId')[["year","round","name_y","name_x","milliseconds_y"]])
pitstop_comb.columns = ['year','round','race','constructor','milliseconds']
pitstop_comb['duration'] = pitstop_comb['milliseconds']/1000


In [40]:
pitstop_comb[pitstop_comb['constructor']=='Red Bull'].sort_values('duration')

,year,round,race,constructor,milliseconds,duration
1380,2012,5,Spanish Grand Prix,Red Bull,13335,13.335
1745,2012,13,Italian Grand Prix,Red Bull,15008,15.008
5825,2017,8,Azerbaijan Grand Prix,Red Bull,15135,15.135
8262,2021,6,Azerbaijan Grand Prix,Red Bull,15277,15.277
4169,2015,10,Hungarian Grand Prix,Red Bull,16453,16.453
...,...,...,...,...,...,...
5495,2016,20,Brazilian Grand Prix,Red Bull,2008600,2008.600
5490,2016,20,Brazilian Grand Prix,Red Bull,2009643,2009.643
8387,2021,10,British Grand Prix,Red Bull,2076405,2076.405
9158,2022,10,British Grand Prix,Red Bull,3034772,3034.772


In [63]:
pitstop_comb.groupby('constructor').agg({'duration':'median'}).reset_index().sort_values('duration')

,constructor,duration
9,Lotus,22.6320
16,Red Bull,22.8220
20,Virgin,22.9390
14,Mercedes,22.9490
5,Ferrari,23.0255
6,Force India,23.3380
13,McLaren,23.3630
10,Lotus F1,23.4210
17,Renault,23.5550
19,Toro Rosso,23.5840
